# DATACHALLENGE

Pour ce datachallenge, nous aurons besoin des modules Python ci-dessous, il vous faut donc évidemment exécuter cette première cellule.

In [22]:
%matplotlib inline
import pandas as pd
import numpy as np
import sklearn

Les différentes méthodes dont nous aurons besoins

### Pour importer les fichiers csv
pd.read_csv

### Pour créer nos echantillons de train et de test
from sklearn import model_selection.train_test_split

model_selection.train_test_split

### Pour calculer une regression linéaire
from sklearn import linear_model

linear_model.LogisticRegression

### Pour faire une PCA
from sklearn import decomposition

decomposition.PCA

### Classificatuer gaussien
from sklearn.naive_bayes import GaussianNB

### Classificateur k-voisins
from sklearn import neighbors

neighbors.KNeighborsClassifier

### Regression logistique
from sklearn import linear_model

linear_model.LogisticRegression

### Regression logistique lasso
from sklearn import linear_model

linear_model.LogisticRegressionCV

### Decision Tree
sklearn.tree.DecisionTreeClassifier

### Bagged Tree
sklearn.tree.DecisionTreeRegressor

### Trouver la meilleur valeur d'un paramètre
form sklearn import model_selection

model_selection.GridSearchCV

### Créer une répartition d'échantillon de test aléatoire
from sklearn import model_selection

model_selection.KFold



# Importation des données

Nous allons importer toutes les données qui vont nous être utiles, même celles qui ne seront pas utiles avant la fin


In [23]:
XTrain_Path = "Xtrain_hgcGIrA.csv"
YTrain_Path = "Ytrain_yL5OjS4.csv"
XTest_Path = "Xtest.csv"

In [24]:
XTrain  =    pd.read_csv(XTrain_Path).to_numpy()
YTrain  =    pd.read_csv(YTrain_Path).to_numpy()
XTest  =    pd.read_csv(XTest_Path).to_numpy()

In [25]:
XTrain

array([['2019-01-07', 1, 0, ..., 0.201, 0.138, 0.091],
       ['2019-01-08', 1, 0, ..., 0.204, 0.152, 0.106],
       ['2019-01-10', 1, 0, ..., 0.213, 0.153, 0.111],
       ...,
       ['2019-05-15', 9, 0, ..., 0.099, 0.129, nan],
       ['2019-03-21', 9, 0, ..., 0.074, 0.101, nan],
       ['2019-01-30', 9, 0, ..., 0.289, 0.354, nan]], dtype=object)

# Découpage train / test

En apprentissage statistique, classiquement un prédicteur est ajusté sur une partie seulement des données et l'erreur de ce dernier est ensuite évaluée sur une autre partie des données disponibles. Ceci permet de ne pas utiliser les mêmes données pour ajuster et évaluer la qualité d'un prédicteur. Cette problématique est l'objet du prochain chapitre.

> En utilisant la fonction [`train_test_split`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split) de la librairie [`sklearn.model_selection`](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection), sélectionner aléatoirement 60% des observations pour l'échantillon d'apprentissage et garder le reste pour l'échantillon de test. 

In [26]:
from sklearn import model_selection

ntot = XTrain.size

XTrain_train,XTrain_test,YTrain_train, YTrain_test = model_selection.train_test_split(XTrain, YTrain, test_size=0.4, train_size=0.6, random_state=0, shuffle=True, stratify=None)